In [358]:
import pdfplumber
import re

# location = r"C:\Users\SamsonC\Documents\Accounting\Accounting_AR\invoice_pdf\8713.pdf"
# location = r"C:\Users\SamsonC\Documents\Accounting\Accounting_AR\Selfish\Misc\5230.pdf"
# 
location = r"C:\Users\SamsonC\Documents\Accounting\Accounting_AR\invoice_pdf\46859.pdf"

# location = r"C:\Users\SamsonC\Documents\Accounting\Accounting_AR\Selfish\Misc\5428.pdf"



In [359]:
# open and read line from pdf in text
with pdfplumber.open(location) as pdf:
    text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())

    print(text)

print()
print("---=-- line splitted --------")
lines = text.split("\n")  
print(lines)
    




---=-- line splitted --------
['']


In [360]:
# Extract table rows
table_row_regex = r'(.+):\s+(\d+)\s+([\d,.]+)\s+([\d,.]+)'
matches = re.findall(table_row_regex, text)

# Print table data
for match in matches:
    description, qty, rate, price = match
    print(f"Description: {description.strip()}")
    print(f"Qty: {qty}")
    print(f"Rate: {rate}")
    print(f"Price: ${price}")

# Extract tax information
tax_match = re.search(r'HST On Sales ([\d.]+%) ([\d,.]+)', text)
if tax_match:
    tax_rate = tax_match.group(1)
    tax_amount = tax_match.group(2)
    print(f"Tax Rate: {tax_rate}")
    print(f"Tax Amount: ${tax_amount}")

In [361]:
# start extracting each invoice item

# extract Date
date_match = re.search(r'Date:\s*(\d{4}-\d{2}-\d{2})', text)
date = date_match.group(1) if date_match else "Date not found"
print(f"Date: {date}")

# extract Invoice Number
invoice_match = re.search(r'Invoice No:\s*(\d+)', text)
invoice_number = invoice_match.group(1) if invoice_match else "Invoice No not found"
print(f"Invoice No: {invoice_number}")

# extract Client Name
client_match = re.search(r'Client:\s*([A-Za-z\s.,-]+)', text)
client_name = client_match.group(1).strip() if client_match else "Client not found"
print(f"Client: {client_name}")

print()

# extract Client Address
extracted_address = None

# for i, line in enumerate(lines):
#     if client_name in line:
#         if i + 1 < len(lines):  # ensure there's a next line
#             next_line = lines[i + 1].strip()
#             # address validation pattern (assumes addresses start with a number)
#             if re.match(r"^\d{1,5}\s[\w\s]+[,\s]*$", next_line):
#                 extracted_address = next_line.split(',')[0].strip()
#                 break

# print(f"Address: {extracted_address if extracted_address else 'No address found'}")

for i, line in enumerate(lines):
    if "Client:" in line:
        if i + 1 < len(lines):
            next_line = lines[i + 1].strip()
            # Take only the first part (street address) before the comma
            extracted_address = next_line.split(',')[0].strip()
        break
print(f"Addres2: {extracted_address if extracted_address else 'No address found'}")

unit = re.search(r"(?i)(?:Unit|Suite)\s*\d+", text, re.MULTILINE)
if unit:
    unit_line = unit.group(0).strip()
    print("Unit:", unit_line)


building_match = re.search(r"(?i)(?:Bldg\.?|Building)\s*[A-Za-z0-9]+", text)
if building_match:
    building_field = building_match.group(0).strip()
    print("Building:", building_field)
else:
    print("Building not found.")
    

city_test = re.search(r"(?m)^\s*[A-Za-z\s]+(?=,\s*[Oo][Nn])", text)
if city_test:
    print("City:", city_test.group(0))
else:
    # print("City not matched.")
    if not re.search(r"(?m)^[A-Za-z\s]+(?=, ON)", text):
        for line in lines:
            # Check if the line contains a postal code pattern (Canadian format) and "Canada"
            if re.search(r"[A-Z]\d[A-Z]\s*\d[A-Z]\d", line) and "Canada" in line:
                parts = line.split(',')
                if len(parts) >= 2:
                    city_test = parts[0].strip()
                    print("City:", city_test)
                break
province_match = re.search(r",\s*([A-Za-z]{2})(?=\s)", text, re.IGNORECASE)
if province_match:
    province = province_match.group(1).upper()
    print("Province22:", province)

# Use a regex that works for various cases:
province_match = re.search(
    r",\s*([A-Za-z]+(?:\s+[A-Za-z]+)*)(?=\s+(?:Canada|[A-Z]\d[A-Z]\s*\d[A-Z]\d|Agreement No:)|\s*$)",
    text,
    re.IGNORECASE
)
if province_match:
    province_extracted = province_match.group(1).strip()
    # Normalize the extracted province to uppercase for comparison.
    province_extracted_upper = province_extracted.upper()
    
    # If the extracted province is the full name "ONTARIO", convert it to "ON".
    if province_extracted_upper == "ONTARIO" or province_extracted_upper == "ONTARIO CANADA":
        province = "ON"
    else:
        province = province_extracted_upper

    print("Province:", province)
else:
    print("Province not matched.")


# province_test = re.search(r",\s*([A-Za-z]{2})(?=\s)", text, re.IGNORECASE)

# if province_test:
#     if province_test.group(0) == "HS":
#         if re.search(r"[A-Z]\d[A-Z]\s*\d[A-Z]\d", line) and "Canada" in line:
#             parts = line.split(',')
#             if len(parts) >= 2:
#                 # Remove extra words after the province name (like 'Canada')
#                 province_tokens = parts[1].strip().split()
#                 if province_tokens:
#                     province = province_tokens[0].strip().upper()
#                     print("Province:", province_test)
#     else:
#         province = province_test.group(1).upper()
#         print("Province:", province)
# else:
#     print("Province not matched.")

# for line in lines:
    # Look for a line with the expected pattern: "City, Province Canada PostalCode"
# if re.search(r"[A-Z]\d[A-Z]\s*\d[A-Z]\d", line) and "Canada" in line:
#     parts = line.split(',')
#     if len(parts) >= 2:
#         # Remove extra words after the province name (like 'Canada')
#         province_tokens = parts[1].strip().split()
#         if province_tokens:
#             province_match = province_tokens[0].strip()
#             print("Alternative Province:", province_match)
    # break




postal_test = re.search(r"[A-Z]\d[A-Z]\s*\d[A-Z]\d", text)
if postal_test:
    print("Postal Code:", postal_test.group(0))
else:
    print("Postal code not matched.")

print()

# extract header with Agreement No and Client (updated for format "#A0429-DE - GO Towns")
header_match = re.search(
    r'(?:Re:\s*)?#([A-Z0-9-]+)\s*-\s*([A-Za-z0-9\s-]+?)(?=\s+Qty\s+Rate\s+Price|\s*$)',
    text
)
if header_match:
    agreement_number = header_match.group(1).strip()
    client_name = header_match.group(2).strip()
    print(f"Agreement No: {agreement_number}")
    print(f"Client: {client_name}")
    print()
else:
    print("Header not found.")



Date: Date not found
Invoice No: Invoice No not found
Client: Client not found

Addres2: No address found
Building not found.
Province not matched.
Postal code not matched.

Header not found.


In [362]:

# Extract only the relevant table section (from "Terms: Net 30" to "HST On Sales")
start_idx = next((i for i, line in enumerate(lines) if "Terms: Net 30" in line), None)
end_idx = next((i for i, line in enumerate(lines) if "HST On Sales" in line), None)

if start_idx is not None and end_idx is not None:
    table_rows = lines[start_idx:end_idx]  # Extract relevant section
else:
    table_rows = []

# Process Table Data
table_data = []
i = 0

while i < len(table_rows):
    row = table_rows[i].strip()

    # Match rows where numbers (Qty, Rate, Price) appear at the end
    match = re.search(r'(.+?)\s+(\d*\.?\d+)\s+([\d,.]+)\s+([\d,.]+)-?', row)
    if match:
        description = match.group(1).strip()
        qty = match.group(2)
        rate = match.group(3)
        price = match.group(4)

        # Check if the next line continues the description (not a new row)
        next_line_index = i + 1
        while next_line_index < len(table_rows):
            next_line = table_rows[next_line_index].strip()
            if not re.search(r'^\d*\.?\d+\s+[\d,.]+\s+[\d,.]+-?$', next_line):  # If next line is NOT a new row
                description += f" {next_line}"
                next_line_index += 1
            else:
                break

        table_data.append((description, qty, rate, price))
        i = next_line_index  # Skip processed lines
    else:
        i += 1  # Continue searching

# Extract HST Row Separately
hst_match = re.search(r'HST On Sales\s+([\d.]+%)\s+([\d,.]+)', text)
if hst_match:
    hst_rate = hst_match.group(1)
    hst_price = hst_match.group(2)
    table_data.append(("HST On Sales", "-", hst_rate, hst_price))  # No qty

# Print Extracted Data
if table_data:
    print("\nExtracted Table Data:")
    for description, qty, rate, price in table_data:
        print(f"Description: {description.strip()}")
        print(f"Qty: {qty}")
        print(f"Rate: {rate}")
        print(f"Price: {price}")
else:
    print("No table data found.")

No table data found.
